In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['Simhei']  #显示中文
plt.rcParams['axes.unicode_minus']=False    #显示负号 

In [2]:
def closest_fraction(value,total_min,total_max):
    # 计算最接近 vaule 的整数比，整数之和在一定范围内
    total_min=int(total_min)+1
    total_max=int(total_max)
    difference = value
    i1=0
    j1=0
    for i in range(1,total_max):
        for j in range(max(total_min-i-1,1),total_max-i+1):
            # print(i,j)
            if abs(i/j-value)<difference:
                difference=abs(i/j-value)
                i1=i
                j1=j
    return i1,j1,difference
print(closest_fraction(0.73,30,100))
print(closest_fraction(0.73,100,100))

(27, 37, 0.0002702702702702675)
(42, 58, 0.00586206896551722)


In [3]:
df=pd.read_excel('Exp8-pre.xlsx')
df

,Distance-Para/cm,Shield/(g/cm^2),Count,Background,Distance-Para/cm.1,Adding-Shield/(g/cm^2),Shield/(g/cm^2).1,Count.1,Background.1,Time(s)
0,25.0,0.0,8574.0,944.0,21,0.000,0.000,2982,994.0,10.0
1,23.0,0.0,4484.0,NaN,21,1.840,1.840,2696,NaN,NaN
2,21.0,0.0,2982.0,NaN,21,1.818,3.658,2248,NaN,NaN
3,19.0,0.0,2362.0,NaN,21,1.761,5.419,2137,NaN,NaN
4,17.0,0.0,1916.0,NaN,21,1.820,7.239,1873,NaN,NaN
5,15.0,0.0,1624.0,NaN,21,1.746,8.985,1729,NaN,NaN
6,NaN,NaN,NaN,NaN,21,1.792,10.777,1615,NaN,NaN


# 不同距离

In [4]:
# 读取数据
para_dis = df['Distance-Para/cm'].dropna().values
count_total_dis = df['Count'].dropna().values
count_back_dis = df['Background'][0]
time = df['Time(s)'][0]
df_dis = pd.DataFrame()
df_dis['总计数-预实验']=count_total_dis
df_dis['本底-预实验']=count_back_dis
df_dis

,总计数-预实验,本底-预实验
0,8574.0,944.0
1,4484.0,944.0
2,2982.0,944.0
3,2362.0,944.0
4,1916.0,944.0
5,1624.0,944.0


In [5]:
df_dis['距离参数']=para_dis
# 计算时间比值和优质因子
r_dis = count_total_dis/count_back_dis
r_t_dis = np.sqrt(r_dis)
df_dis['理想时间比值']=r_t_dis

Q_dis = (1+r_dis**(1/2))**2/((count_back_dis/time)*(r_dis-1)**2)
df_dis['优质因子']=Q_dis
df_dis

,总计数-预实验,本底-预实验,距离参数,理想时间比值,优质因子
0,8574.0,944.0,25.0,3.013740,0.002612
1,4484.0,944.0,23.0,2.179449,0.007615
2,2982.0,944.0,21.0,1.777329,0.017531
3,2362.0,944.0,19.0,1.581809,0.031294
4,1916.0,944.0,17.0,1.424662,0.058741
5,1624.0,944.0,15.0,1.311617,0.109090


In [6]:
# 计算目标精度的最少的测量数据
nu = 0.05
T_dis = Q_dis/nu**2
T_dis
df_dis[f'误差{nu}的总测量时间']=T_dis
df_dis

,总计数-预实验,本底-预实验,距离参数,理想时间比值,优质因子,误差0.05的总测量时间
0,8574.0,944.0,25.0,3.013740,0.002612,1.044916
1,4484.0,944.0,23.0,2.179449,0.007615,3.045996
2,2982.0,944.0,21.0,1.777329,0.017531,7.012587
3,2362.0,944.0,19.0,1.581809,0.031294,12.517789
4,1916.0,944.0,17.0,1.424662,0.058741,23.496424
5,1624.0,944.0,15.0,1.311617,0.109090,43.636097


In [7]:
# T_max=max(120,T_dis.max())
# 取最大测量时间为120 计算时间分配
T_max = 100
T_min = 80

ratio_dis=[]
for i in range(len(T_dis)):
    ratio_dis.append(closest_fraction(r_t_dis[i],T_min,T_max))

df_dis['最佳总计数测量时间']=np.array(ratio_dis)[:,0]
df_dis['最佳本地测量时间']=np.array(ratio_dis)[:,1]
df_dis['比值误差']=np.array(ratio_dis)[:,2]
df_dis['测量结果相对误差']=np.sqrt(Q_dis/(np.array(ratio_dis)[:,0]+np.array(ratio_dis)[:,1]))

# df_dis.to_excel('Exp8-pre.xlsx',sheet_name='dis_result')

df_dis

,总计数-预实验,本底-预实验,距离参数,理想时间比值,优质因子,误差0.05的总测量时间,最佳总计数测量时间,最佳本地测量时间,比值误差,测量结果相对误差
0,8574.0,944.0,25.0,3.013740,0.002612,1.044916,60.0,20.0,0.013740,0.005714
1,4484.0,944.0,23.0,2.179449,0.007615,3.045996,61.0,28.0,0.000878,0.009250
2,2982.0,944.0,21.0,1.777329,0.017531,7.012587,64.0,36.0,0.000449,0.013241
3,2362.0,944.0,19.0,1.581809,0.031294,12.517789,49.0,31.0,0.001164,0.019778
4,1916.0,944.0,17.0,1.424662,0.058741,23.496424,57.0,40.0,0.000338,0.024608
5,1624.0,944.0,15.0,1.311617,0.109090,43.636097,55.0,42.0,0.002093,0.033536


In [14]:
with pd.ExcelWriter('Exp8-pre_result.xlsx') as writer:
    df_dis.to_excel(writer,sheet_name='distance result')

# 屏蔽层

In [8]:
para_she = df['Shield/(g/cm^2).1'].values
count_total_she = df['Count.1'].values
count_back_she = df['Background.1'][0]

df_she = pd.DataFrame()
df_she['总计数-预实验']=count_total_she
df_she['本底-预实验']=count_back_she
df_she

,总计数-预实验,本底-预实验
0,2982,994.0
1,2696,994.0
2,2248,994.0
3,2137,994.0
4,1873,994.0
5,1729,994.0
6,1615,994.0


In [9]:
df_she['屏蔽参数']=para_she
# 计算时间比值和优质因子
r_she = count_total_she/count_back_she
r_t_she = np.sqrt(r_she)
df_she['理想时间比值']=r_t_she

Q_she = (1+r_she**(1/2))**2/((count_back_she/time)*(r_she-1)**2)

df_she['优质因子']=Q_she
df_she

,总计数-预实验,本底-预实验,屏蔽参数,理想时间比值,优质因子
0,2982,994.0,0.000,1.732051,0.018773
1,2696,994.0,1.840,1.646898,0.024040
2,2248,994.0,3.658,1.503852,0.039629
3,2137,994.0,5.419,1.466254,0.046277
4,1873,994.0,7.239,1.372700,0.072426
5,1729,994.0,8.985,1.318877,0.098939
6,1615,994.0,10.777,1.274656,0.133363


In [10]:
nu = 0.05
T_she = Q_she/nu**2
T_she
# 计算目标精度的最少的测量数据

df_she[f'误差{nu}的总测量时间']=T_she
df_she

,总计数-预实验,本底-预实验,屏蔽参数,理想时间比值,优质因子,误差0.05的总测量时间
0,2982,994.0,0.000,1.732051,0.018773,7.509157
1,2696,994.0,1.840,1.646898,0.024040,9.616161
2,2248,994.0,3.658,1.503852,0.039629,15.851430
3,2137,994.0,5.419,1.466254,0.046277,18.510977
4,1873,994.0,7.239,1.372700,0.072426,28.970391
5,1729,994.0,8.985,1.318877,0.098939,39.575566
6,1615,994.0,10.777,1.274656,0.133363,53.345122


In [11]:
# T_max=max(120,T_she.max())
T_max = 100
# 取最大测量时间为120 计算时间分配

ratio_she=[]
for i in range(len(T_she)):
    ratio_she.append(closest_fraction(r_t_she[i],T_min,T_max))

df_she['最佳总计数测量时间']=np.array(ratio_she)[:,0]
df_she['最佳本地测量时间']=np.array(ratio_she)[:,1]
df_she['比值误差']=np.array(ratio_she)[:,2]
df_she['测量结果相对误差']=np.sqrt(Q_she/(np.array(ratio_she)[:,0]+np.array(ratio_she)[:,1]))

# df_dis.to_excel('Exp8-pre.xlsx',sheet_name='she_result')
df_she

,总计数-预实验,本底-预实验,屏蔽参数,理想时间比值,优质因子,误差0.05的总测量时间,最佳总计数测量时间,最佳本地测量时间,比值误差,测量结果相对误差
0,2982,994.0,0.000,1.732051,0.018773,7.509157,52.0,30.0,0.001283,0.015131
1,2696,994.0,1.840,1.646898,0.024040,9.616161,56.0,34.0,0.000161,0.016344
2,2248,994.0,3.658,1.503852,0.039629,15.851430,48.0,32.0,0.003852,0.022257
3,2137,994.0,5.419,1.466254,0.046277,18.510977,50.0,34.0,0.004335,0.023472
4,1873,994.0,7.239,1.372700,0.072426,28.970391,48.0,35.0,0.001272,0.029540
5,1729,994.0,8.985,1.318877,0.098939,39.575566,54.0,41.0,0.001804,0.032272
6,1615,994.0,10.777,1.274656,0.133363,53.345122,51.0,40.0,0.000344,0.038282


In [15]:
with pd.ExcelWriter('Exp8-pre_result.xlsx',mode='a',engine='openpyxl') as writer:
    df_dis.to_excel(writer,sheet_name='shield result')